In [1]:
import pandas as pd
import numpy as np
import clickhouse_connect
from elasticsearch import Elasticsearch

# GET PINPP 

### DEFENDANT

In [29]:
def get_pinpp_of_defendant_with_limit(limit=800):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    query = f"""
SELECT OriginAirportID, FlightDate  FROM ontime
WHERE toDate(FlightDate) >= '2022-03-31'
ORDER BY FlightDate DESC
limit {limit}
    """
    client = clickhouse_connect.get_client(
        host=host,
        port=port,  
        user=user,
        password=password,
        secure=True  
    )
    query = query
    result = client.query(query)
    client.close()
    

    return result.result_rows

result=get_pinpp_of_defendant_with_limit()

df_def=pd.DataFrame(result, columns=['pinfl', 'FlightDate'])
df_def['jinoyatchimi']=1

pinfl_list_to_check = list(df_def['pinfl'])

In [33]:
def get_all_pinpp_of_defendant():
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer"
    password = ""

    query = f"""
    SELECT login FROM actors  
 """
    
    client = clickhouse_connect.get_client(host=host, port=port, user=user, password=password)
    result = client.query(query)
    client.close()

    return result.result_rows

In [ ]:
df_allpinpp_of_defendant = 

In [31]:
df_def['pinfl'].value_counts()

pinfl
11298    111
11057    108
14107     67
13303     53
14100     42
        ... 
14307      1
14321      1
10849      1
10792      1
10994      1
Name: count, Length: 85, dtype: int64

### NORMAL HUMANS

In [3]:
def get_pinpp_of_normal_humans(not_get_pinfl_list, limit=1000):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""
    excluded_pinpps = ", ".join([f"'{pinpp}'" for pinpp in not_get_pinfl_list])
    
    query = f"""
    SELECT OriginAirportID
    FROM ontime
    WHERE OriginAirportID NOT IN ({excluded_pinpps})
    LIMIT {limit}
    """

    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True 
    )
    
    result = client.query(query)
    client.close() 
    return result.result_rows

result=get_pinpp_of_normal_humans(pinfl_list_to_check, limit=1000)

df_normal=pd.DataFrame(result, columns=['pinfl'])
df_normal['jinoyatchimi']=0

In [4]:
df = pd.concat([df_def, df_normal], ignore_index=True).sample(frac = 1)
pinpp_list_overall = list(df['pinfl'])

In [16]:
import clickhouse_connect
from typing import List, Tuple, Optional

def get_data_by_pinpp(
    pinpp_list_overall: List[str],
    chunk_size: int = 100,
    limit: Optional[int] = 1000
) -> List[Tuple]:
        config = {
            "host": "play.clickhouse.com",
            "port": 443,
            "user": "explorer",
            "password": "",
            "secure": True
        }
        
        with clickhouse_connect.get_client(**config) as client:
            result = []
            
            chunks = [
                pinpp_list_overall[i:i + chunk_size] 
                for i in range(0, len(pinpp_list_overall), chunk_size)
            ]
            
            for chunk in chunks:
                query = """
                SELECT Year, Reporting_Airline, Month
                FROM ontime
                WHERE OriginAirportID IN ({})
                """
                
                if limit:
                    query += f" LIMIT {limit}"
                
                placeholders = ", ".join([f"'{pinpp}'" for pinpp in chunk])
                formatted_query = query.format(placeholders)
                
                partial_result = client.query(formatted_query)
                result.extend(partial_result.result_rows)
            
            return result

In [24]:
pinpp_list = pinfl_list_to_check
results = get_data_by_pinpp(
    pinpp_list_overall=pinpp_list,
    chunk_size=50,
    limit=500
)

df_overall_data = pd.DataFrame(results, columns=['Year', 'Reporting_Airline', 'Month'])

In [28]:
df_overall_data.sort_values(by='Year', ascending=True)

,Year,Reporting_Airline,Month
0,2000,US,3
341,2000,US,3
340,2000,US,3
339,2000,US,3
338,2000,US,3
...,...,...,...
159,2000,US,3
158,2000,US,3
157,2000,US,3
170,2000,US,3


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       11000 non-null  int64 
 1   1       11000 non-null  object
 2   2       11000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 257.9+ KB


In [15]:
df.head()

,0,1,2
0,1996,AA,4
1,1996,AA,4
2,1996,AA,4
3,1996,AA,4
4,1996,AA,4


### GET normal humans pinpp is not in pinpp of defendants

In [ ]:
def get_normal_human_pinpp(limit=50000):
    host = "play.clickhouse.com" 
    port = 443
    user = "explorer" 
    password = ""

    client = clickhouse_connect.get_client(host, port, user, password)
    